In [2]:
!pip install mysql-connector-python==8.0.33

In [3]:
import mysql.connector
from tkinter import *
from tkinter import messagebox, ttk

# Database Connection
mydb = mysql.connector.connect(
    host="localhost", user='your_user', password="your_password", database='Parking', charset='utf8')
mycursor = mydb.cursor()

# Helper Function: Create Entry Fields and Labels
def create_form(window, labels):
    entries = []
    for i, label_text in enumerate(labels):
        ttk.Label(window, text=label_text).grid(row=i, column=0, padx=10, pady=5)
        entry = ttk.Entry(window)
        entry.grid(row=i, column=1, padx=10, pady=5)
        entries.append(entry)
    return entries

# Save Functions
def save_parking_detail(entries, window):
    try:
        data = (
            int(entries[0].get()), entries[1].get(), entries[2].get(), 
            entries[3].get(), entries[4].get(), int(entries[5].get())
        )
        payment = {1: 20, 2: 30, 3: 50, 4: 80, 5: 100, 6: 200}.get(data[5], 0)
        mycursor.execute(
            "INSERT INTO parkmaster12 (pid, pnm, level, freespace, vehicleno, nod, payment) VALUES (%s, %s, %s, %s, %s, %s, %s)", 
            data + (payment,)
        )
        mydb.commit()
        messagebox.showinfo("Success", "Parking details added successfully!")
        window.destroy()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def save_vehicle_detail(entries, window):
    try:
        data = (int(entries[0].get()), entries[1].get(), entries[2].get())
        mycursor.execute("INSERT INTO vehicle (vid, vnm, dateofpur) VALUES (%s, %s, %s)", data)
        mydb.commit()
        messagebox.showinfo("Success", "Vehicle details added successfully!")
        window.destroy()
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Create Forms
def create_parking_form():
    add_window = Toplevel(root, bg="#D8DEE9")
    add_window.title("Add Parking Detail")
    add_window.geometry("400x300")
    labels = ["Parking ID", "Parking Name", "Level", "Free Space (YES/NO)", "Vehicle Number", "Number of Days"]
    entries = create_form(add_window, labels)
    ttk.Button(add_window, text="Save",command=lambda: save_parking_detail(entries, add_window)).grid(row=6, column=1, pady=10)

def create_vehicle_form():
    add_window = Toplevel(root, bg="#D8DEE9")
    add_window.title("Add Vehicle Detail")
    add_window.geometry("400x200")
    labels = ["Vehicle Number", "Vehicle Name/Model", "Date of Purchase (YYYY-MM-DD)"]
    entries = create_form(add_window, labels)
    ttk.Button(add_window, text="Save",command=lambda: save_vehicle_detail(entries, add_window)).grid(row=3, column=1, pady=10)


# Remove Vehicle and Parking Details
def remove_vehicle_record():
    remove_window = Toplevel(root)
    remove_window.configure(bg="#D8DEE9")
    remove_window.title("Remove Vehicle Record")  # Set title
    remove_window.geometry("300x150")  # Set geometry

    ttk.Label(remove_window, text="Vehicle Number").grid(row=0, column=0, padx=10, pady=5)
    vid_entry = ttk.Entry(remove_window)
    vid_entry.grid(row=0, column=1)

    def delete_vehicle():
        try:
            vid = int(vid_entry.get())

            # Delete vehicle from vehicle table
            sql_vehicle = "DELETE FROM vehicle WHERE vid = %s"
            mycursor.execute(sql_vehicle, (vid,))

            # Delete corresponding parking details from parkmaster12
            sql_parking = "DELETE FROM parkmaster12 WHERE vehicleno = %s"
            mycursor.execute(sql_parking, (vid,))

            mydb.commit()  # Commit both deletions
            messagebox.showinfo("Success", "Vehicle and its parking details removed!")
            remove_window.destroy()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    # Add the delete button below the entry field
    ttk.Button(remove_window, text="Delete", command=delete_vehicle).grid(row=1, column=1, pady=10)

# View Parking Details
def view_parking_detail():
    mycursor.execute("SELECT * FROM parkmaster12")  # Execute the query
    records = mycursor.fetchall()  # Fetch all results

    view_window = Toplevel(root)
    view_window.title("View Parking Details")
    view_window.geometry("900x400")

    tree = ttk.Treeview(view_window, columns=(1, 2, 3, 4, 5, 6, 7), show="headings", height=10)
    headings = ["Parking ID", "Parking Name", "Level", "Free Space", "Vehicle No", "Days", "Payment"]

    for idx, heading in enumerate(headings):
        tree.heading(idx + 1, text=heading)
        tree.column(idx + 1, width=80, anchor='center')

    for record in records:
        tree.insert('', 'end', values=record)

    tree.pack(fill=BOTH, expand=True)


# Fetch Vehicle and Parking Detail
def view_vehicle_detail():
    view_vehicle_window = Toplevel(root)
    view_vehicle_window.configure(bg="#D8DEE9")
    view_vehicle_window.title("View Vehicle Detail")
    view_vehicle_window.geometry("400x200")

    ttk.Label(view_vehicle_window, text="Enter Vehicle Number").grid(row=0, column=0, padx=10, pady=5)
    vid_entry = ttk.Entry(view_vehicle_window)
    vid_entry.grid(row=0, column=1)

    def fetch_vehicle_and_parking_detail():
        try:
            vid = int(vid_entry.get())
            sql = """
                SELECT v.vid, v.vnm, v.dateofpur, p.pid, p.pnm, p.level, p.freespace, p.nod, p.payment
                FROM vehicle v
                LEFT JOIN parkmaster12 p ON v.vid = p.vehicleno
                WHERE v.vid = %s
            """
            mycursor.execute(sql, (vid,))
            record = mycursor.fetchone()

            if record:
                details = f"Vehicle: {record[1]} (No: {record[0]})\nPurchase Date: {record[2]}\n" + \
                          f"Parking ID: {record[3] or 'N/A'} at {record[4] or 'N/A'}\nLevel: {record[5]}\nDays: {record[7]}\nPayment: Rs.{record[8]}"
                messagebox.showinfo("Details", details)
            else:
                messagebox.showerror("Error", "Vehicle not found!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    ttk.Button(view_vehicle_window, text="Fetch", command=fetch_vehicle_and_parking_detail).grid(row=1, column=1, pady=10)

# Main GUI Setup
root = Tk()
root.title("Parking Management System")
root.geometry("750x500")
root.configure(bg="#4C566A")

Label(root, text="Parking Management System", font=("Verdana", 24, "bold"), bg="#2c3e50", fg="#ecf0f1").pack(pady=20)

# Button Frame
button_frame = ttk.Frame(root, padding="20 10 20 10")
button_frame.pack(pady=30)

buttons = [
    ("Add Parking Detail", create_parking_form),
    ("View Parking Details", view_parking_detail),
    ("Add Vehicle Detail", create_vehicle_form),
    ("Remove Vehicle Record", remove_vehicle_record),
    ("View Vehicle Detail", view_vehicle_detail)
]

for i, (text, command) in enumerate(buttons):
    ttk.Button(button_frame, text=text, command=command).grid(row=i, column=0, pady=5)

root.mainloop()